Parsing Data via Python (might not even need this)

First Vegalite Vis

In [4]:
import altair as alt
import pandas as pd

# Create a dataframe
data = pd.DataFrame({
    'x': [(i % 9) + 1 for i in range(81)],
    'y': [(i // 9) + 1 for i in range(81)],
    'value': [i * 5 for i in range(81)]
})

# Create the heatmap
heatmap = alt.Chart(data).mark_rect().encode(
    x=alt.X('x:O', title='Column', axis=alt.Axis(tickCount=9, grid=False)),
    y=alt.Y('y:O', title='Row', axis=alt.Axis(tickCount=9, grid=False)),
    color=alt.Color('value:Q', title='Value', scale=alt.Scale(scheme='viridis'))
).properties(
    width=400,
    height=400,
    title='Ultimate Tic Tac Toe Heat Map'
)

# Display the heatmap
heatmap

alt.Chart(...)

In [8]:
import json
import pandas as pd
import altair as alt

# Load telemetry data from JSON file
with open('telemetry (5).json') as f:
    telemetry_data = json.load(f)

# Extract cell notations that completed smaller boards
completed_cells = [
    event['details']['cellNotation']
    for event in telemetry_data
    if event['eventType'] == 'completedSmallerBoard'
]

# Create a dataframe with the frequency of cell choices
cell_counts = pd.DataFrame(completed_cells, columns=['cellNotation'])
cell_counts['frequency'] = 1
cell_counts = cell_counts.groupby('cellNotation').sum().reset_index()

# Normalize the frequencies
total_events = cell_counts['frequency'].sum()
cell_counts['probability'] = cell_counts['frequency'] / total_events

# Expand to all cell notations (1-9) to ensure all cells are represented
all_cells = pd.DataFrame({'cellNotation': range(1, 10)})
cell_counts = pd.merge(all_cells, cell_counts, on='cellNotation', how='left').fillna(0)

# Create x and y coordinates for the heatmap
cell_counts['x'] = (cell_counts['cellNotation'] - 1) % 3 + 1
cell_counts['y'] = (cell_counts['cellNotation'] - 1) // 3 + 1

# Create the heatmap
heatmap = alt.Chart(cell_counts).mark_rect().encode(
    x=alt.X('x:O', title='Column', axis=alt.Axis(tickCount=3, grid=False)),
    y=alt.Y('y:O', title='Row', axis=alt.Axis(tickCount=3, grid=False)),
    color=alt.Color('probability:Q', title='Probability', scale=alt.Scale(scheme='viridis'))
).properties(
    width=300,
    height=300,
    title='Most Common Cells to End Small Boards'
)

# Display the heatmap
heatmap


alt.Chart(...)

In [1]:
import json
import pandas as pd
import altair as alt

# Load telemetry data from JSON file
with open('telemetry (5).json') as f:
    telemetry_data = json.load(f)

# Extract moves from telemetry data
moves = [
    (event['details']['largerBoardNotation'], event['details']['cellNotation'])
    for event in telemetry_data
    if event['eventType'] == 'move'
]

# Map the moves to the overall 9x9 grid positions
def map_to_grid(largerBoardNotation, cellNotation):
    # Larger board positions
    board_row = (largerBoardNotation - 1) // 3
    board_col = (largerBoardNotation - 1) % 3
    # Cell positions
    cell_row = (cellNotation - 1) // 3
    cell_col = (cellNotation - 1) % 3
    # Overall positions in the 9x9 grid
    grid_row = board_row * 3 + cell_row + 1
    grid_col = board_col * 3 + cell_col + 1
    return grid_row, grid_col

# Apply the mapping
grid_positions = [map_to_grid(move[0], move[1]) for move in moves]

# Create a dataframe with the frequency of cell choices
df = pd.DataFrame(grid_positions, columns=['y', 'x'])
df['frequency'] = 1
df = df.groupby(['y', 'x']).sum().reset_index()

# Normalize the frequencies to get probabilities
total_moves = df['frequency'].sum()
df['probability'] = df['frequency'] / total_moves

# Ensure all cells are represented
all_cells = pd.DataFrame([(i, j) for i in range(1, 10) for j in range(1, 10)], columns=['y', 'x'])
df = pd.merge(all_cells, df, on=['y', 'x'], how='left').fillna(0)

# Create the heatmap
heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X('x:O', title='Column', axis=alt.Axis(tickCount=9, grid=False)),
    y=alt.Y('y:O', title='Row', axis=alt.Axis(tickCount=9, grid=False)),
    color=alt.Color('probability:Q', title='Probability', scale=alt.Scale(scheme='viridis'))
).properties(
    width=400,
    height=400,
    title='Most Common Squares Played in UTTT by Random Bot(Normalized)'
)

# Display the heatmap
heatmap


alt.Chart(...)